In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2024-02-01T14:58:59.723282-08:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.21.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.0-1042-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 224
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: Install the gpu extra (pip install 'watermark[gpu]') to display GPU information for NVIDIA chipsets



In [4]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import optuna
import gc
import logging

/home/dgxuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%watermark --iversions

xgboost: 2.0.3
numpy  : 1.26.3
optuna : 3.5.0
pandas : 1.5.3
logging: 0.5.1.2
dask   : 2023.11.0



In [6]:
!nvidia-smi -L

GPU 0: NVIDIA H100 80GB HBM3 (UUID: GPU-5c583ee7-8fb1-b26a-4bea-16e45d984a32)
GPU 1: NVIDIA H100 80GB HBM3 (UUID: GPU-a6ab06f5-a6e2-18c1-9dec-0dcd29f44a46)
GPU 2: NVIDIA H100 80GB HBM3 (UUID: GPU-bb8d5098-3c56-c48d-a0a3-fcdfcec6d3f5)
GPU 3: NVIDIA H100 80GB HBM3 (UUID: GPU-cdbe686b-1611-999b-8e8d-a3c5f35b40c4)
GPU 4: NVIDIA H100 80GB HBM3 (UUID: GPU-0df1cef0-fc95-cc88-b5f4-239889b3acba)
GPU 5: NVIDIA H100 80GB HBM3 (UUID: GPU-9dca657c-dbe4-08b7-fe7c-5653f183f0b6)
GPU 6: NVIDIA H100 80GB HBM3 (UUID: GPU-33389782-e2ad-5022-997d-cf470313879c)
GPU 7: NVIDIA H100 80GB HBM3 (UUID: GPU-926eaa05-87fb-35e0-0a98-adb2ad41d0be)


In [7]:
cluster = LocalCUDACluster(n_workers=8)
client = Client(cluster)

In [8]:
!nvidia-smi

Thu Feb  1 14:59:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:1B:00.0 Off |                    0 |
| N/A   27C    P0              67W / 700W |      4MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [56]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [10]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_oho_fold_{i}.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_oho_fold_{i}.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [11]:
train = pd.read_csv('../input/train.csv.zip')

target = train['target'].values
target

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['hist']),  # 'gpu_hist','hist'
        'device': trial.suggest_categorical('device',['cuda']),
        'lambda': trial.suggest_float('lambda',1e-3,10.0, log=True),
        'alpha': trial.suggest_float('alpha',1e-3,10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.001,0.1, log=True),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    gini = gini_normalized(target, train_oof)
    
    return gini

In [13]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_5.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_porto_5.db", study_name="five_fold_optuna_xgb_5", direction='maximize')

In [14]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

[14:59:48] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[14:59:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

[15:00:10] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[15:00:17] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

[15:00:24] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[15:00:31] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

[15:00:38] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[15:00:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

[15:00:51] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[15:00:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

[15:01:04] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[15:01:15] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

[15:01:22]

CPU times: user 39.7 s, sys: 15.4 s, total: 55.1 s
Wall time: 4min 8s


In [15]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_device,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.261091,9.320316,0.477649,cuda,logloss,0.074662,0.001820,16,96,binary:logistic,0.524617,hist,COMPLETE
1,1,0.286779,0.626090,0.617290,cuda,logloss,0.014329,0.007502,19,64,binary:logistic,0.548255,hist,COMPLETE
2,2,0.271957,0.147410,0.453876,cuda,logloss,0.001651,0.002307,14,222,binary:logistic,0.770649,hist,COMPLETE


In [16]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_5.csv', index=False)
df.head(8)

[15:04:08] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[15:04:16] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-01 15:04:17,678 - distributed.utils_perf - WARNING - full garbage collections took 71% CPU time recently (threshold: 10%)
2024-02-01 15:04:17,736 - distributed.utils_perf - WARNING - full garbage collections took 71% CPU time recently (threshold: 10%)
[15:04:23] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[15:04:31] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-01 15:04:32,474 - distributed.utils_perf - WARNING - full garbage collections took 71% CPU time recently (threshold: 10%)
2024-02-01 15:04:32,528 - distributed.utils_perf - WARNING - full garbage collections took 71% CPU time recently (threshold: 10%)
[15:04:38] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[15:04:45] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } 

CPU times: user 1min 4s, sys: 16.6 s, total: 1min 20s
Wall time: 6min 14s


,number,value,params_alpha,params_colsample_bytree,params_device,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.261091,9.320316,0.477649,cuda,logloss,0.074662,0.001820,16,96,binary:logistic,0.524617,hist,COMPLETE
1,1,0.286779,0.626090,0.617290,cuda,logloss,0.014329,0.007502,19,64,binary:logistic,0.548255,hist,COMPLETE
2,2,0.271957,0.147410,0.453876,cuda,logloss,0.001651,0.002307,14,222,binary:logistic,0.770649,hist,COMPLETE
3,3,0.287178,0.186078,0.790183,cuda,logloss,0.141901,0.015706,10,229,binary:logistic,0.718290,hist,COMPLETE
4,4,0.274219,0.115748,0.739794,cuda,logloss,0.123462,0.047147,24,209,binary:logistic,0.444212,hist,COMPLETE
5,5,0.219778,0.152644,0.780991,cuda,logloss,0.795898,0.057804,23,99,binary:logistic,0.972187,hist,COMPLETE
6,6,0.273179,5.072753,0.462350,cuda,logloss,0.014858,0.002766,19,134,binary:logistic,0.691930,hist,COMPLETE
7,7,0.286466,0.003027,0.341739,cuda,logloss,2.955274,0.011005,17,185,binary:logistic,0.486281,hist,COMPLETE


In [17]:
study.best_trial.params

{'objective': 'binary:logistic',
 'tree_method': 'hist',
 'device': 'cuda',
 'lambda': 0.14190076165750204,
 'alpha': 0.1860780014774321,
 'colsample_bytree': 0.7901829038590573,
 'subsample': 0.7182899032077363,
 'learning_rate': 0.015706484870345004,
 'max_depth': 10,
 'min_child_weight': 229,
 'eval_metric': 'logloss'}

In [18]:
df.value.max()

0.2871776897327792

In [ ]:
%%time
study.optimize(objective, n_trials=500)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_5.csv', index=False)
df.head(20)

[15:10:47] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[15:10:59] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-01 15:11:00,666 - distributed.utils_perf - WARNING - full garbage collections took 73% CPU time recently (threshold: 10%)
2024-02-01 15:11:00,730 - distributed.utils_perf - WARNING - full garbage collections took 73% CPU time recently (threshold: 10%)
[15:11:06] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[15:11:18] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-01 15:11:19,793 - distributed.utils_perf - WARNING - full garbage collections took 73% CPU time recently (threshold: 10%)
2024-02-01 15:11:19,857 - distributed.utils_perf - WARNING - full garbage collections took 73% CPU time recently (threshold: 10%)
[15:11:25] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[15:11:36] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } 

In [20]:
df.value.max()

0.2895797316411886

In [21]:
study.best_trial.params

{'objective': 'binary:logistic',
 'tree_method': 'hist',
 'device': 'cuda',
 'lambda': 0.016329246014877414,
 'alpha': 3.33029065395022,
 'colsample_bytree': 0.5456389940628341,
 'subsample': 0.7684743978358726,
 'learning_rate': 0.014586087705844453,
 'max_depth': 7,
 'min_child_weight': 135,
 'eval_metric': 'logloss'}

In [27]:
best_params = study.best_trial.params

In [31]:
test = delayed(pd.read_csv)(f'../input/X_test_df.csv.zip')
test = dd.from_delayed(test)
test = test[test.columns.difference(['id'])]

In [32]:
test.shape

(Delayed('int-54936ab0-f805-41aa-aaee-2750e5bf29f8'), 217)

In [33]:
test.head()

,col_0,col_1,col_10,col_100,col_101,col_102,col_103,col_104,col_105,col_106,...,col_90,col_91,col_92,col_93,col_94,col_95,col_96,col_97,col_98,col_99
0,0.0,8.0,0.5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,5.0,0.9,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,3.0,0.4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,6.0,0.1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,7.0,0.9,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
dtest = xgb.dask.DaskDMatrix(client, test.values, enable_categorical=True)

In [35]:
%%time
num_round = 1000
train_oof = np.zeros((target.shape[0],))
test_preds = 0

kf = KFold(5, shuffle=True, random_state=137)

for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        print(i)
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, best_params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        val_predictions = xgb.dask.predict(client, booster, dval)
        val_predictions = val_predictions.compute()

        test_predictions = xgb.dask.predict(client, booster, dtest)
        test_predictions = test_predictions.compute()
    
        train_oof[val_index] = val_predictions
        test_preds += test_predictions/5
    
        del dtrain, dval, output
        gc.collect()
        gc.collect()

gini = gini_normalized(target, train_oof)
print(gini)

0


[07:34:31] task [xgboost.dask-7]:tcp://127.0.0.1:41419 got new rank 0
[07:34:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 07:34:44,263 - distributed.utils_perf - WARNING - full garbage collections took 80% CPU time recently (threshold: 10%)
2024-02-02 07:34:44,537 - distributed.utils_perf - WARNING - full garbage collections took 81% CPU time recently (threshold: 10%)


1


[07:34:49] task [xgboost.dask-4]:tcp://127.0.0.1:38713 got new rank 0
[07:34:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 07:35:02,674 - distributed.utils_perf - WARNING - full garbage collections took 81% CPU time recently (threshold: 10%)
2024-02-02 07:35:02,967 - distributed.utils_perf - WARNING - full garbage collections took 81% CPU time recently (threshold: 10%)


2


...

3


[07:35:26] task [xgboost.dask-0]:tcp://127.0.0.1:40795 got new rank 0
[07:35:32] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 07:35:37,341 - distributed.utils_perf - WARNING - full garbage collections took 81% CPU time recently (threshold: 10%)
2024-02-02 07:35:37,627 - distributed.utils_perf - WARNING - full garbage collections took 81% CPU time recently (threshold: 10%)


4


[07:35:43] task [xgboost.dask-3]:tcp://127.0.0.1:38175 got new rank 0
[07:35:50] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 07:35:55,330 - distributed.utils_perf - WARNING - full garbage collections took 81% CPU time recently (threshold: 10%)
2024-02-02 07:35:55,620 - distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (threshold: 10%)


0.2895797316411886
CPU times: user 6.55 s, sys: 3.47 s, total: 10 s
Wall time: 1min 29s


In [36]:
test_preds

array([0.02825447, 0.02195566, 0.02305012, ..., 0.03104293, 0.02074758,
       0.0294082 ], dtype=float32)

In [38]:
test_preds.min()

0.006525852

In [40]:
train_oof.max()

0.27948254346847534

In [41]:
train_oof.min()

0.006003982853144407

In [42]:
submission = pd.read_csv('../input/sample_submission.csv.zip')

In [43]:
submission['target'] = test_preds
submission.head()

,id,target
0,0,0.028254
1,1,0.021956
2,2,0.023050
3,3,0.013815
4,4,0.035183


In [44]:
submission.to_csv('../submissions/best_xgb_oho_optuna_h100_0.csv', index=False)

0.2849 public, 0.29105 private

In [45]:
%%time
num_round = 1000
n_seeds = 5
n_folds = 5

train_oof = np.zeros((target.shape[0],))
test_preds = 0

kf = KFold(n_folds, shuffle=True, random_state=137)

for i, (train_index, val_index) in enumerate(kf.split(train,target)):
    for jj in range(n_seeds):

        best_params['random_state'] = 3*jj**2+1777
    
        print(i)
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, best_params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        val_predictions = xgb.dask.predict(client, booster, dval)
        val_predictions = val_predictions.compute()

        test_predictions = xgb.dask.predict(client, booster, dtest)
        test_predictions = test_predictions.compute()
    
        train_oof[val_index] += val_predictions/n_seeds
        test_preds += test_predictions/(n_folds*n_seeds)
    
        del dtrain, dval, output
        gc.collect()
        gc.collect()

gini = gini_normalized(target, train_oof)
print(gini)

0


[07:46:49] task [xgboost.dask-4]:tcp://127.0.0.1:38713 got new rank 0
[07:46:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 07:47:02,391 - distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (threshold: 10%)
2024-02-02 07:47:02,785 - distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (threshold: 10%)


0


[07:47:08] task [xgboost.dask-1]:tcp://127.0.0.1:45615 got new rank 0
[07:47:16] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 07:47:21,265 - distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (threshold: 10%)
2024-02-02 07:47:21,655 - distributed.utils_perf - WARNING - full garbage collections took 71% CPU time recently (threshold: 10%)


0


...

4


[07:53:52] task [xgboost.dask-7]:tcp://127.0.0.1:41419 got new rank 0
[07:53:59] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 07:54:04,253 - distributed.utils_perf - WARNING - full garbage collections took 97% CPU time recently (threshold: 10%)
2024-02-02 07:54:04,635 - distributed.utils_perf - WARNING - full garbage collections took 97% CPU time recently (threshold: 10%)


4


[07:54:10] task [xgboost.dask-4]:tcp://127.0.0.1:38713 got new rank 0
[07:54:17] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 07:54:22,199 - distributed.utils_perf - WARNING - full garbage collections took 97% CPU time recently (threshold: 10%)
2024-02-02 07:54:22,599 - distributed.utils_perf - WARNING - full garbage collections took 97% CPU time recently (threshold: 10%)


TypeError: 'numpy.float64' object is not callable

In [48]:
train_oof.max()

0.2905692458152771

In [52]:
gini(target, train_oof)

TypeError: 'numpy.float64' object is not callable

In [55]:
train_oof

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [50]:
train_oof.astype(np.float32)

array([0.04382015, 0.03149953, 0.01924309, ..., 0.01722216, 0.02299856,
       0.01950087], dtype=float32)

In [58]:
gini_normalized(target, train_oof)

0.2892946965464846

In [59]:
submission['target'] = test_preds
submission.head()

,id,target
0,0,0.027935
1,1,0.022870
2,2,0.023015
3,3,0.013951
4,4,0.035421


In [60]:
submission.to_csv('../submissions/best_xgb_oho_optuna_h100_1.csv', index=False)

0.28486 public, 0.29113 private

In [61]:
%%time
num_round = 1000
n_seeds = 10
n_folds = 5

train_oof = np.zeros((target.shape[0],))
test_preds = 0

kf = KFold(n_folds, shuffle=True, random_state=137)

for i, (train_index, val_index) in enumerate(kf.split(train,target)):
    print(i)
    for jj in range(n_seeds):

        best_params['random_state'] = 3*jj**2+1777
    
        print(jj)
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, best_params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        val_predictions = xgb.dask.predict(client, booster, dval)
        val_predictions = val_predictions.compute()

        test_predictions = xgb.dask.predict(client, booster, dtest)
        test_predictions = test_predictions.compute()
    
        train_oof[val_index] += val_predictions/n_seeds
        test_preds += test_predictions/(n_folds*n_seeds)
    
        del dtrain, dval, output
        gc.collect()
        gc.collect()

gini = gini_normalized(target, train_oof)
print(gini)

0
0


[08:05:02] task [xgboost.dask-7]:tcp://127.0.0.1:41419 got new rank 0
[08:05:10] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 08:05:15,523 - distributed.utils_perf - WARNING - full garbage collections took 85% CPU time recently (threshold: 10%)
2024-02-02 08:05:15,925 - distributed.utils_perf - WARNING - full garbage collections took 85% CPU time recently (threshold: 10%)


1


[08:05:21] task [xgboost.dask-4]:tcp://127.0.0.1:38713 got new rank 0
[08:05:29] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 08:05:34,519 - distributed.utils_perf - WARNING - full garbage collections took 85% CPU time recently (threshold: 10%)
2024-02-02 08:05:34,914 - distributed.utils_perf - WARNING - full garbage collections took 85% CPU time recently (threshold: 10%)


2


...

8


[08:19:54] task [xgboost.dask-7]:tcp://127.0.0.1:41419 got new rank 0
[08:20:01] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 08:20:06,465 - distributed.utils_perf - WARNING - full garbage collections took 97% CPU time recently (threshold: 10%)
2024-02-02 08:20:06,869 - distributed.utils_perf - WARNING - full garbage collections took 97% CPU time recently (threshold: 10%)


9


[08:20:12] task [xgboost.dask-4]:tcp://127.0.0.1:38713 got new rank 0
[08:20:19] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

2024-02-02 08:20:24,728 - distributed.utils_perf - WARNING - full garbage collections took 97% CPU time recently (threshold: 10%)
2024-02-02 08:20:25,132 - distributed.utils_perf - WARNING - full garbage collections took 97% CPU time recently (threshold: 10%)


0.2893686664415629
CPU times: user 1min 24s, sys: 29.8 s, total: 1min 53s
Wall time: 15min 28s


In [62]:
submission['target'] = test_preds
submission.to_csv('../submissions/best_xgb_oho_optuna_h100_2.csv', index=False)
submission.head()

,id,target
0,0,0.027973
1,1,0.022714
2,2,0.023101
3,3,0.013864
4,4,0.035546


0.28491 public, 0.2911 private